In [ ]:
## DAGAN UResNetGerenetor

In [1]:
import tensorflow as tf

slim = tf.contrib.slim


C:\Users\home\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
input = tf.zeros(shape=[16,28,28,3])

In [3]:
z_input = tf.random_normal([16, 100], mean=0, stddev=1)

In [4]:
def upscale(x, h_size, w_size):
        """
        Upscales an image using nearest neighbour
        :param x: Input image
        :param h_size: Image height size
        :param w_size: Image width size
        :return: Upscaled image
        """
        [b, h, w, c] = [int(dim) for dim in x.get_shape()]

        return tf.image.resize_nearest_neighbor(x, (h_size, w_size))

In [5]:
def count_parameters(network_variables, name):
    """
    This method counts the total number of unique parameters for a list of variable objects
    :param network_variables: A list of tf network variable objects
    :param name: Name of the network
    """
    total_parameters = 0
    for variable in network_variables:
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        variable_parametes = 1
        for dim in shape:
            variable_parametes *= dim.value

        total_parameters += variable_parametes
    print(name, total_parameters)

In [ ]:
tf.reset_default_graph()

In [ ]:
ge = UResnetGenerator(is_training=True, scope='g')

In [ ]:
ge(input)

In [6]:
class UResnet(object):
    
    def __init__(self, layer_sizes, layer_padding, batch_size, num_channels=1,inner_layers=0, name="g"):
        
        '''
        Initialize a UResNet generator.
        :param layer_sizes: A list with the filter sizes for each MultiLayer e.g. [64, 64, 128, 128]
        :param layer_padding: A list with the padding type for each layer e.g. ["SAME", "SAME", "SAME", "SAME"]
        :param batch_size: An integer indicating the batch size
        :param num_channels: An integer indicating the number of input channels
        :param inner_layers: An integer indicating the number of inner layers per MultiLayer
        '''
        self.reuse = False
        self.batch_size = batch_size
        self.num_channels = num_channels
        self.layer_sizes = layer_sizes
        self.layer_padding = layer_padding
        self.inner_layers = inner_layers
        self.conv_layer_num = 0
        self.build = True
        self.name = name
        '''
        input = [batch_size, h, w, num_chanels] of tensor, [batch_size, 28, 28, 3]
        '''
        

    
    def __call__(self, z_input, input, training=False):
        with slim.arg_scope([slim.conv2d, slim.conv2d_transpose],
                           num_outputs = 64,
                            padding = 'SAME',
                            kernel_size = [3,3],
                            stride = (1,1),
                            activation_fn = None):
            with slim.arg_scope([slim.batch_norm],
                               decay =0.99, scale= True, center=True, is_training=training, renorm=True):

                with tf.variable_scope('fisrt_single_conv'): 
                    conv = slim.conv2d(input, stride=(2,2))
                    conv = tf.nn.leaky_relu(conv)
                    conv = slim.batch_norm(conv) # [batch_size, 14, 14, 3]

                with tf.variable_scope('first_resnet_block'):
                    input_projection = slim.conv2d(input, num_outputs=input.get_shape()[3], stride=(2,2))
                    output1 = tf.concat([conv, input_projection], axis=3)
                    print(output1)

                    conv1_1 = slim.conv2d(output1)
                    conv1_1 = tf.nn.leaky_relu(conv1_1)
                    conv1_1 = slim.batch_norm(conv1_1)

                    output2 = tf.concat([conv, conv1_1], axis=3)
                    print(output2)

                    conv1_2 = slim.conv2d(output2)
                    conv1_2 = tf.nn.leaky_relu(conv1_2)
                    conv1_2 = slim.batch_norm(conv1_2)

                    output3 = tf.concat([conv1_2, output2], axis=3)
                    print(output3)

                    conv1_3 = slim.conv2d(output3)
                    conv1_3 = tf.nn.leaky_relu(conv1_3)
                    conv1_3 = slim.batch_norm(conv1_3)

                    output4 = tf.concat([conv1_3, output3], axis=3)
                    print(output4)

                    conv1_4 = slim.conv2d(output4, stride=(2,2))
                    conv1_4 = tf.nn.leaky_relu(conv1_4)
                    conv1_4 = slim.batch_norm(conv1_4)
                    conv1_4 = slim.dropout(conv1_4, keep_prob= 0.5) # [batch_size, 7, 7, 64]

                with tf.variable_scope('second_resnet_block'):
                    input_projection = slim.conv2d(conv1_3, num_outputs=conv1_3.get_shape()[3], stride=(2,2))
                    print(input_projection)
                    output5 = tf.concat([conv1_4, input_projection], axis=3)
                    print(output5)

                    conv2_1 = slim.conv2d(output5)
                    conv2_1 = tf.nn.leaky_relu(conv2_1)
                    conv2_1 = slim.batch_norm(conv2_1) #x4

                    output6 = tf.concat([output5, conv2_1], axis=3)
                    print(output6)

                    conv2_2 = slim.conv2d(output6)
                    conv2_2 = tf.nn.leaky_relu(conv2_2)
                    conv2_2 = slim.batch_norm(conv2_2)

                    output7 = tf.concat([conv2_2, output6], axis=3)
                    print(output7)

                    conv2_3 = slim.conv2d(output7)
                    conv2_3 = tf.nn.leaky_relu(conv2_3)
                    conv2_3 = slim.batch_norm(conv2_3)

                    output8 = tf.concat([conv2_3, output7], axis=3)

                    conv2_4 = slim.conv2d(output8, stride=(2,2))
                    conv2_4 = tf.nn.leaky_relu(conv2_4)
                    conv2_4 = slim.batch_norm(conv2_4)
                    conv2_4 = slim.dropout(conv2_4, keep_prob= 0.5) # [batch_size, 4, 4, 64]
                    print(conv2_4)

                with tf.variable_scope('third_resnet_block'):
                    input_projection = slim.conv2d(conv2_3, num_outputs=conv2_3.get_shape()[3], stride=(2,2)) #[batch_size, 4, 4, 3]
                    print(input_projection)
                    output9 = tf.concat([conv2_4, input_projection], axis=3)
                    print(output9)

                    conv3_1 = slim.conv2d(output9)
                    conv3_1 = tf.nn.leaky_relu(conv3_1)
                    conv3_1 = slim.batch_norm(conv3_1)

                    output3_1 = tf.concat([output9, conv3_1], axis=3)
                    print(output3_1)

                    conv3_2 = slim.conv2d(output3_1)
                    conv3_2 = tf.nn.leaky_relu(conv3_2)
                    conv3_2 = slim.batch_norm(conv3_2)

                    output3_2 = tf.concat([conv3_2, output3_1], axis=3)
                    print(output3_2)

                    conv3_3 = slim.conv2d(output3_2)
                    conv3_3 = tf.nn.leaky_relu(conv3_3)
                    conv3_3 = slim.batch_norm(conv3_3)

                    output3_3 = tf.concat([conv3_3, output3_2], axis=3)

                    conv3_4 = slim.conv2d(output3_3, stride=(2,2))
                    conv3_4 = tf.nn.leaky_relu(conv3_4)
                    conv3_4 = slim.batch_norm(conv3_4)
                    conv3_4 = slim.dropout(conv3_4, keep_prob= 0.5) # [batch_size, 2, 2, 64]
                    conv3_4.get_shape()
                    print(conv3_4)
                    print('end of encoder')

            ##Decoder    
                with tf.variable_scope('first_decoder_block'):
                    z_dense = tf.layers.dense(z_input, 2*2*8) #num_filters set 8
                    z_noise = tf.reshape(z_dense, [input.shape[0], 2, 2, 8])
                    d_output1 = tf.concat([conv3_4, z_noise], axis=3) #[batch_size, 2, 2, 72]
                    print('first decoder', d_output1)

                    d_conv1_1 = slim.conv2d(d_output1)
                    d_conv1_1 = tf.nn.leaky_relu(d_conv1_1)
                    d_conv1_1 = slim.batch_norm(d_conv1_1)

                    d_output1_1 = tf.concat([d_conv1_1, d_output1], axis=3) #[b, 2, 2, 136]

                    d_conv1_2 = slim.conv2d(d_output1_1)
                    d_conv1_2 = tf.nn.leaky_relu(d_conv1_2)
                    d_conv1_2 = slim.batch_norm(d_conv1_2)

                    d_output1_2 = tf.concat([d_conv1_2, d_output1_1], axis=3) #[b, 2, 2, 200]

                    d_conv1_3 = slim.conv2d(d_output1_2)
                    d_conv1_3 = tf.nn.leaky_relu(d_conv1_3)
                    d_conv1_3 = slim.batch_norm(d_conv1_3)

                    d_output1_3 = tf.concat([d_conv1_3, d_output1_2], axis=3) #[b, 2, 2, 264]

                    d_conv1_4 = upscale(d_output1_3, 4, 4)
                    d_conv1_4 = slim.conv2d_transpose(d_conv1_4)
                    d_conv1_4 = tf.nn.leaky_relu(d_conv1_4)
                    d_conv1_4 = slim.batch_norm(d_conv1_4)
                    d_conv1_4 = slim.dropout(d_conv1_4, keep_prob= 0.5)

                    d_output1_4 = tf.concat([d_conv1_4, conv2_4], axis = 3) #[b, 4, 4, 128]


                with tf.variable_scope('second_decoder_block'):
                    z_dense = tf.layers.dense(z_input, 4*4*4) #num_filters set 4
                    z_noise = tf.reshape(z_dense, [input.shape[0], 4, 4, 4])
                    d_output2 = tf.concat([d_output1_4, z_noise], axis=3) #[b, 4, 4, 132]

                    input_projection = upscale(d_conv1_3, 4, 4)
                    input_projection = slim.conv2d_transpose(input_projection, num_outputs=d_conv1_3.get_shape()[3]) #[batch_size, 4, 4, 64]
                    print(input_projection)
                    d_output2 = tf.concat([d_output2, input_projection], axis=3) #[b, 4, 4, 196]
                    print(d_output2)

                    d_conv2_1 = slim.conv2d(d_output2)
                    d_conv2_1 = tf.nn.leaky_relu(d_conv2_1)
                    d_conv2_1 = slim.batch_norm(d_conv2_1)

                    d_output2_1 = tf.concat([d_conv2_1, d_output2], axis=3) #[b, 4, 4, 260]

                    d_conv2_2 = slim.conv2d(d_output2_1)
                    d_conv2_2 = tf.nn.leaky_relu(d_conv2_2)
                    d_conv2_2 = slim.batch_norm(d_conv2_2)

                    d_output2_2 = tf.concat([d_conv2_2, d_output2_1], axis=3) #[b, 4, 4, 324]

                    d_conv2_3 = slim.conv2d(d_output2_2)
                    d_conv2_3 = tf.nn.leaky_relu(d_conv2_3)
                    d_conv2_3 = slim.batch_norm(d_conv2_3)

                    d_output2_3 = tf.concat([d_conv2_3, d_output2_2], axis=3)

                    d_conv2_4 = upscale(d_output2_3, 7, 7)
                    d_conv2_4 = slim.conv2d_transpose(d_conv2_4)
                    d_conv2_4 = tf.nn.leaky_relu(d_conv2_4)
                    d_conv2_4 = slim.batch_norm(d_conv2_4)
                    d_conv2_4 = slim.dropout(d_conv2_4, keep_prob= 0.5)

                    d_output2_4 = tf.concat([d_conv2_4, conv1_4], axis = 3) #[b, 7, 7, 128]


                with tf.variable_scope('third_decoder_block'):
                    z_dense = tf.layers.dense(z_input, 7*7*2) #num_filters set 2
                    z_noise = tf.reshape(z_dense, [input.shape[0], 7, 7, 2])
                    d_output3 = tf.concat([d_output2_4, z_noise], axis=3) #[b, 4, 4, 130]

                    input_projection = upscale(d_conv2_3, 7, 7)
                    input_projection = slim.conv2d_transpose(input_projection, num_outputs=d_conv2_3.get_shape()[3]) #[batch_size, 4, 4, 64]
                    print(input_projection)
                    d_output3 = tf.concat([d_output3, input_projection], axis=3) #[b, 4, 4, 196]
                    print(d_output3)

                    d_conv3_1 = slim.conv2d(d_output3)
                    d_conv3_1 = tf.nn.leaky_relu(d_conv3_1)
                    d_conv3_1 = slim.batch_norm(d_conv3_1)

                    d_output3_1 = tf.concat([d_conv3_1, d_output3], axis=3) #[b, 4, 4, 260]

                    d_conv3_2 = slim.conv2d(d_output3_1)
                    d_conv3_2 = tf.nn.leaky_relu(d_conv3_2)
                    d_conv3_2 = slim.batch_norm(d_conv3_2)

                    d_output3_2 = tf.concat([d_conv3_2, d_output3_1], axis=3) #[b, 4, 4, 324]

                    d_conv3_3 = slim.conv2d(d_output3_2)
                    d_conv3_3 = tf.nn.leaky_relu(d_conv3_3)
                    d_conv3_3 = slim.batch_norm(d_conv3_3)

                    d_output3_3 = tf.concat([d_conv3_3, d_output3_2], axis=3)

                    d_conv3_4 = upscale(d_output3_3, 14, 14)
                    d_conv3_4 = slim.conv2d_transpose(d_conv3_4)
                    d_conv3_4 = tf.nn.leaky_relu(d_conv3_4)
                    d_conv3_4 = slim.batch_norm(d_conv3_4)
                    d_conv3_4 = slim.dropout(d_conv3_4, keep_prob= 0.5) #[b, 14, 14, 64]

                    d_output3_4 = tf.concat([d_conv3_4, conv], axis = 3) #[b, 14 14, 128]

                with tf.variable_scope('forth_decoder_block'):

                    input_projection = upscale(d_conv3_3, 14, 14)
                    input_projection = slim.conv2d_transpose(input_projection, num_outputs=d_conv3_3.get_shape()[3]) #[batch_size, 4, 4, 64]
                    print(input_projection)
                    d_output4 = tf.concat([d_output3_4, input_projection], axis=3) #[b, 14, 14, 192]
                    print(d_output4)

                    d_conv4_1 = slim.conv2d(d_output4)
                    d_conv4_1 = tf.nn.leaky_relu(d_conv4_1)
                    d_conv4_1 = slim.batch_norm(d_conv4_1)

                    d_output4_1 = tf.concat([d_conv4_1, d_output4], axis=3) #[b, 14, 14, 256]

                    d_conv4_2 = slim.conv2d(d_output4_1)
                    d_conv4_2 = tf.nn.leaky_relu(d_conv4_2)
                    d_conv4_2 = slim.batch_norm(d_conv4_2)

                    d_output4_2 = tf.concat([d_conv4_2, d_output4_1], axis=3) #[b, 14, 14, 320]

                    d_conv4_3 = upscale(d_output4_2, 28, 28)
                    d_conv4_3 = slim.conv2d_transpose(d_conv4_3)
                    d_conv4_3 = tf.nn.leaky_relu(d_conv4_3)
                    d_conv4_3 = slim.batch_norm(d_conv4_3)
                    d_conv4_3 = slim.dropout(d_conv4_3, keep_prob= 0.5) #[b, 28, 28, 64]

                    d_output4_3 = tf.concat([d_conv4_3, input], axis=3) #[b, 28, 28, 67]


                with tf.variable_scope('last_block'):

                    input_projection = upscale(d_conv4_3, 28, 28)
                    input_projection = slim.conv2d_transpose(input_projection, num_outputs=d_conv4_3.get_shape()[3]) #[b, 28, 28, 64]
                    print(input_projection)
                    d_output5 = tf.concat([d_output4_3, input_projection], axis=3) #[b, 14, 14, 127]
                    print(d_output5)

                    d_conv5_1 = slim.conv2d(d_output5)
                    d_conv5_1 = tf.nn.leaky_relu(d_conv5_1)
                    d_conv5_1 = slim.batch_norm(d_conv5_1)

                    d_output5_1 = tf.concat([d_conv5_1, d_output5], axis=3) #[b, 14, 14, 195]

                    d_conv5_2 = slim.conv2d(d_output5_1)
                    d_conv5_2 = tf.nn.leaky_relu(d_conv5_2)
                    d_conv5_2 = slim.batch_norm(d_conv5_2)

                    d_output5_2 = tf.concat([d_conv5_2, d_output5_1], axis=3) #[b, 14, 14, 259]

                    #In p process
                    d_conv5_3 = slim.conv2d(d_output5_2)
                    d_conv5_3 = tf.nn.leaky_relu(d_conv5_3)
                    d_conv5_3 = slim.batch_norm(d_conv5_3)

                    d_conv5_4 = slim.conv2d(d_conv5_3, num_outputs = 3)
                    d_conv5_4 = tf.nn.leaky_relu(d_conv5_4)
                    d_conv5_4 = slim.batch_norm(d_conv5_4)

                with tf.variable_scope('g_tanh'):
                    gan_decoder = tf.tanh(d_conv5_4, name='outputs')
                    
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self.name)

        if self.build:
            count_parameters(self.variables, 'generator_parameter_num')
        self.build = False
        
        return gan_decoder
                
    

In [7]:
u = UResnet(0, 0, 0)

In [8]:
print(u)

In [9]:
u(z_input, input)

Tensor("first_resnet_block/concat:0", shape=(16, 14, 14, 67), dtype=float32)
Tensor("first_resnet_block/concat_1:0", shape=(16, 14, 14, 128), dtype=float32)
Tensor("first_resnet_block/concat_2:0", shape=(16, 14, 14, 192), dtype=float32)
Tensor("first_resnet_block/concat_3:0", shape=(16, 14, 14, 256), dtype=float32)
Tensor("second_resnet_block/Conv/BiasAdd:0", shape=(16, 7, 7, 64), dtype=float32)
Tensor("second_resnet_block/concat:0", shape=(16, 7, 7, 128), dtype=float32)
Tensor("second_resnet_block/concat_1:0", shape=(16, 7, 7, 192), dtype=float32)
Tensor("second_resnet_block/concat_2:0", shape=(16, 7, 7, 256), dtype=float32)
Tensor("second_resnet_block/Dropout/dropout_1/mul:0", shape=(16, 4, 4, 64), dtype=float32)
Tensor("third_resnet_block/Conv/BiasAdd:0", shape=(16, 4, 4, 64), dtype=float32)
Tensor("third_resnet_block/concat:0", shape=(16, 4, 4, 128), dtype=float32)
Tensor("third_resnet_block/concat_1:0", shape=(16, 4, 4, 192), dtype=float32)
Tensor("third_resnet_block/concat_2:0", 

<tf.Tensor 'g_tanh/outputs:0' shape=(16, 28, 28, 64) dtype=float32>

In [ ]:
class UResnetGenerator(object):
    def __init__(self, is_training=True, scope=None) :
        self.scope = scope
        self.is_training = is_training
        self.output_channels = 3
        self.batch_norm_params = {'decay' : 0.99, 'scale' : True, 'center' : True,
                                 'is_training' : self.is_training, 'renorm' : True}
        
    
    def encoder_block(self, x1, num): #last output, previous_output 
        with slim.arg_scope([slim.conv2d, slim.conv2d_transpose],
                            num_outputs = 64, padding = 'SAME',
                            kernel_size = [3,3], stride = (1,1),
                            activation_fn = tf.nn.leaky_relu,
                            normalizer_fn=slim.batch_norm, normalizer_params= self.batch_norm_params):

            conv1_1 = slim.conv2d(x1)
            output1_1 = tf.concat([conv1_1, x1], axis=3)
            
            conv1_2 = slim.conv2d(output1_1)
            output1_2 = tf.concat([conv1_2, output1_1], axis=3)
            
            conv1_3 = slim.conv2d(output1_2)
            output1_3 = tf.concat([conv1_3, output1_2], axis=3)
            
            conv1_4 = slim.conv2d(output1_3)
            output = slim.dropout(conv1_4, keep_prob=0.5)
            if num == 3:
                pass
            else :
                input_projection = slim.conv2d(output1_3, num_outputs=output1_3.get_shape()[3], stride=(2,2),
                                               activation_fn= None, normalizer_fn= None)
                output = tf.concat([output, input_projection], axis=3)
            print(output)
            
        return output
    '''
    def decoder_block(self, x1, z_input, num):
        with slim.arg_scope([slim.conv2d, slim.conv2d_transpose],
                            num_outputs = 64, padding = 'SAME',
                            kernel_size = [3,3], stride = (1,1),
                            activation_fn = tf.nn.leaky_relu,
                            normalizer_fn=slim.batch_norm, normalizer_params= self.batch_norm_params):
        return None
        '''
            
    def __call__(self, input, reuse=True):
        with tf.variable_scope('first_single_conv'):
            conv1 = slim.conv2d(input, num_outputs = 64, padding = 'SAME', #[batch_size, 14, 14, 64]
                                 kernel_size = [3,3], stride = (2,2),
                                 activation_fn = tf.nn.leaky_relu,
                                 normalizer_fn=slim.batch_norm, normalizer_params= self.batch_norm_params)

            input_projection = slim.conv2d(input, num_outputs=input.get_shape()[3], kernel_size = [3,3], stride=(2,2),
                                       activation_fn= None, normalizer_fn= None)
            conv1 = tf.concat([conv1, input_projection], axis=3)
            print('first_output ', conv1)

        with tf.variable_scope('Generator_encoder/' + self.scope, reuse=reuse) as scope:
            self.en1 = self.encoder_block(conv1, 1) #[b, 7, 7, 128]
            self.en2 = self.encoder_block(en1, 2) #[b, 4, 4, 128]
            self.en3 = self.encoder_block(en2, 3) #[b, 2, 2, 128]

        return en3
                
            